<a href="https://colab.research.google.com/github/feliciahf/NLP-Project/blob/main/amazon/colab_FH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notes

Check whether stemming & lemmatization make difference

Do we need bigram/trigram frequencies (within titles)?

Do we need dataframe including cleaned data & genres? (which cleaned data?) -> create dictionary of lists maybe
https://www.geeksforgeeks.org/python-ways-to-create-a-dictionary-of-lists/

How do we incorporate POS-tagging & NER in classifier?

# Importing & Cleaning data

This section imports the data into a pandas dataframe and goes through the following preprocessing steps:

-Case collapsing 
-Remove punctuation 
-Tokenization 
-N-Grams: bigrams and trigrams 
-Stemming -> check whether this makes a difference 
-Lemmatization -> check whether this makes a difference 
-Part-of-speech (POS) tagging 
-Named entity recognition (NER) 

## Import Data
We first import the data as a pandas dataframe. Then we create a list made up of the title and genre columns from the original data. We also create a list including all 32 genres.

In order to import the data into Google Colab, we have to first upload the csv file:

In [1]:
# mount Google Drive
from google.colab import  drive
drive.mount('/drive')

Mounted at /drive


In [10]:
# import file from Google Drive
import pandas as pd
df = pd.read_csv('/drive/My Drive/book32listing.csv',encoding='latin1', header=None)

In [11]:
# drop columns that are not needed
#df = pd.read_csv("book32listing.csv", encoding='latin1', header=None)
df1 = df[[3,6]] # only columns with titles and genres
df1.columns = ['title', 'genre']
print(df1)

                                                    title      genre
0                         Mom's Family Wall Calendar 2016  Calendars
1                         Doug the Pug 2016 Wall Calendar  Calendars
2       Moleskine 2016 Weekly Notebook, 12M, Large, Bl...  Calendars
3                 365 Cats Color Page-A-Day Calendar 2016  Calendars
4                    Sierra Club Engagement Calendar 2016  Calendars
...                                                   ...        ...
207567  ADC the Map People Washington D.C.: Street Map...     Travel
207568  Washington, D.C., Then and Now: 69 Sites Photo...     Travel
207569  The Unofficial Guide to Washington, D.C. (Unof...     Travel
207570      Washington, D.C. For Dummies (Dummies Travel)     Travel
207571  Fodor's Where to Weekend Around Boston, 1st Ed...     Travel

[207572 rows x 2 columns]


In [12]:
titles = df1['title'] # list of all titles
titles1 = titles.values.tolist() # change to list of strings
print(titles1[0:6]) # test whether it worked

["Mom's Family Wall Calendar 2016", 'Doug the Pug 2016 Wall Calendar', 'Moleskine 2016 Weekly Notebook, 12M, Large, Black, Soft Cover (5 x 8.25)', '365 Cats Color Page-A-Day Calendar 2016', 'Sierra Club Engagement Calendar 2016', 'Sierra Club Wilderness Calendar 2016']


In [13]:
genres = df1['genre']
genres = genres.values.tolist()
genres = pd.DataFrame(genres)

In [14]:
df1.genre.unique() # list of all possible genres

array(['Calendars', 'Comics & Graphic Novels', 'Test Preparation',
       'Mystery, Thriller & Suspense', 'Science Fiction & Fantasy',
       'Romance', 'Humor & Entertainment', 'Literature & Fiction',
       'Gay & Lesbian', 'Engineering & Transportation',
       'Cookbooks, Food & Wine', 'Crafts, Hobbies & Home',
       'Arts & Photography', 'Education & Teaching',
       'Parenting & Relationships', 'Self-Help', 'Computers & Technology',
       'Medical Books', 'Science & Math', 'Health, Fitness & Dieting',
       'Business & Money', 'Law', 'Biographies & Memoirs', 'History',
       'Politics & Social Sciences', 'Reference',
       'Christian Books & Bibles', 'Religion & Spirituality',
       'Sports & Outdoors', 'Teen & Young Adult', "Children's Books",
       'Travel'], dtype=object)

## Case Collapsing
Change all uppercase to lowercase letters

In [15]:
case_collap = map(lambda x:x.lower(), titles1)
case_collap_list = list(case_collap)
print(case_collap_list[0:6])

["mom's family wall calendar 2016", 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook, 12m, large, black, soft cover (5 x 8.25)', '365 cats color page-a-day calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Remove Punctuation
Remove punctuation by creating a translation table. 
Punctuation to be removed is given in string: string.punctuation

In [16]:
import string
trans = str.maketrans('', '', string.punctuation)
rem_punct = [s.translate(trans) for s in case_collap_list]
print(rem_punct[0:6])

['moms family wall calendar 2016', 'doug the pug 2016 wall calendar', 'moleskine 2016 weekly notebook 12m large black soft cover 5 x 825', '365 cats color pageaday calendar 2016', 'sierra club engagement calendar 2016', 'sierra club wilderness calendar 2016']


## Tokenization
This splits all titles into words (output: list of lists of strings)

In [17]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
tokenized_titles = [word_tokenize(i) for i in rem_punct]
print(tokenized_titles[:10])

[['moms', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cats', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016'], ['thomas', 'kinkade', 'the', 'disney', 'dreams', 'collection', '2016', 'wall', 'calendar'], ['ansel', 'adams', '2016', 'wall', 'calendar'], ['dilbert', '2016', 'daytoday', 'calendar'], ['mary', 'engelbreit', '2016', 'deluxe', 'wall', 'calendar', 'never', 'give', 'up']]


## N-Grams: Bigrams and Trigrams
Create bigrams + trigrams (could be done in one go e.g. n=2 for bigrams, n=3 for trigrams etc.)

In [19]:
# bigrams
token_bigram = []
for title in tokenized_titles:
    title_bigram = []
    for w in range(len(title) - 1):
        title_bigram.append([title[w], title[w + 1]])
    token_bigram.append(title_bigram)
print(token_bigram[:6]) # test whether working

[[['moms', 'family'], ['family', 'wall'], ['wall', 'calendar'], ['calendar', '2016']], [['doug', 'the'], ['the', 'pug'], ['pug', '2016'], ['2016', 'wall'], ['wall', 'calendar']], [['moleskine', '2016'], ['2016', 'weekly'], ['weekly', 'notebook'], ['notebook', '12m'], ['12m', 'large'], ['large', 'black'], ['black', 'soft'], ['soft', 'cover'], ['cover', '5'], ['5', 'x'], ['x', '825']], [['365', 'cats'], ['cats', 'color'], ['color', 'pageaday'], ['pageaday', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'engagement'], ['engagement', 'calendar'], ['calendar', '2016']], [['sierra', 'club'], ['club', 'wilderness'], ['wilderness', 'calendar'], ['calendar', '2016']]]


In [20]:
# trigrams
token_trigram = []
for title in tokenized_titles:
    title_trigram = []
    for w in range(len(title) - 2):
        title_trigram.append([title[w], title[w + 1], title[w + 2]])
    token_trigram.append(title_trigram)
print(token_trigram[:6]) # test whether working

[[['moms', 'family', 'wall'], ['family', 'wall', 'calendar'], ['wall', 'calendar', '2016']], [['doug', 'the', 'pug'], ['the', 'pug', '2016'], ['pug', '2016', 'wall'], ['2016', 'wall', 'calendar']], [['moleskine', '2016', 'weekly'], ['2016', 'weekly', 'notebook'], ['weekly', 'notebook', '12m'], ['notebook', '12m', 'large'], ['12m', 'large', 'black'], ['large', 'black', 'soft'], ['black', 'soft', 'cover'], ['soft', 'cover', '5'], ['cover', '5', 'x'], ['5', 'x', '825']], [['365', 'cats', 'color'], ['cats', 'color', 'pageaday'], ['color', 'pageaday', 'calendar'], ['pageaday', 'calendar', '2016']], [['sierra', 'club', 'engagement'], ['club', 'engagement', 'calendar'], ['engagement', 'calendar', '2016']], [['sierra', 'club', 'wilderness'], ['club', 'wilderness', 'calendar'], ['wilderness', 'calendar', '2016']]]


## Stemming
Test this out to see whether it makes a difference in final classifier

PorterStemmer (one algorithm for stemming; less aggressive than LancasterStemming)

We first create an empty list to contain lists of stems in each title. The second empty list will contain the stem of the title. Each stemmed word in the title will be added to the second list. Each time the second list is filled, it will appended as a list to the first list. 

In [21]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()
stems = []   
for title in tokenized_titles:
    stems_title = []
    for word in title:
        stems_title.append(porter.stem(word))
    stems.append(stems_title)
    
print(stems[0:6]) # test whether it works

[['mom', 'famili', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskin', '2016', 'weekli', 'notebook', '12m', 'larg', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engag', 'calendar', '2016'], ['sierra', 'club', 'wilder', 'calendar', '2016']]


## Lemmatization
Same principle as with stemming. 
We should test this out to see whether it makes a difference in final classifier

In [22]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
lemmas = []
for title in tokenized_titles:
    lemmas_title = []
    for word in title:
        lemmas_title.append(lemmatizer.lemmatize(word))
    lemmas.append(lemmas_title)
    
print(lemmas[0:6]) # test whether it works

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[['mom', 'family', 'wall', 'calendar', '2016'], ['doug', 'the', 'pug', '2016', 'wall', 'calendar'], ['moleskine', '2016', 'weekly', 'notebook', '12m', 'large', 'black', 'soft', 'cover', '5', 'x', '825'], ['365', 'cat', 'color', 'pageaday', 'calendar', '2016'], ['sierra', 'club', 'engagement', 'calendar', '2016'], ['sierra', 'club', 'wilderness', 'calendar', '2016']]


## Part-of-Speech (POS) Tagging
Create list of lists with tokens and their corresponding part-of-speech tag in each title

In [23]:
nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag

postag = []
for title in tokenized_titles:
    postag.append(nltk.pos_tag(title))
    
print(postag[0:6]) # testing whether postag worked

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[[('moms', 'NNS'), ('family', 'NN'), ('wall', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('doug', 'VB'), ('the', 'DT'), ('pug', 'NN'), ('2016', 'CD'), ('wall', 'NN'), ('calendar', 'NN')], [('moleskine', 'NN'), ('2016', 'CD'), ('weekly', 'JJ'), ('notebook', 'NN'), ('12m', 'CD'), ('large', 'JJ'), ('black', 'JJ'), ('soft', 'JJ'), ('cover', 'NN'), ('5', 'CD'), ('x', 'JJ'), ('825', 'CD')], [('365', 'CD'), ('cats', 'NNS'), ('color', 'VBP'), ('pageaday', 'IN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('engagement', 'NN'), ('calendar', 'NN'), ('2016', 'CD')], [('sierra', 'NN'), ('club', 'NN'), ('wilderness', 'NN'), ('calendar', 'NN'), ('2016', 'CD')]]


## Named Entity Recognition (NER)
Create list of lists with words, correspondent POS and named entity tags for each title. This uses postags created in previous step

In [ ]:
nltk.download('maxent_ne_chunker')
nltk.download('words')
from nltk import ne_chunk
from nltk.chunk import tree2conlltags

ner = []
for title in titles1:
    ner.append(tree2conlltags(ne_chunk(pos_tag(word_tokenize(title)))))
    
print(ner[0:6]) # test whether NER worked

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[[('Mom', 'NNP', 'B-PERSON'), ("'s", 'POS', 'O'), ('Family', 'NNP', 'B-PERSON'), ('Wall', 'NNP', 'I-PERSON'), ('Calendar', 'NNP', 'I-PERSON'), ('2016', 'CD', 'O')], [('Doug', 'NNP', 'O'), ('the', 'DT', 'O'), ('Pug', 'NNP', 'O'), ('2016', 'CD', 'O'), ('Wall', 'NNP', 'B-FACILITY'), ('Calendar', 'NNP', 'I-FACILITY')], [('Moleskine', 'NN', 'O'), ('2016', 'CD', 'O'), ('Weekly', 'NNP', 'O'), ('Notebook', 'NNP', 'O'), (',', ',', 'O'), ('12M', 'CD', 'O'), (',', ',', 'O'), ('Large', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Black', 'NNP', 'B-PERSON'), (',', ',', 'O'), ('Soft', 'NNP', 'B-PERSON'), ('Cover', 'NNP', 'I-PERSON'), ('(', '(', 'O'), ('5', 'CD', 'O'), ('x', 'RB', 'O'), ('8.25', 'CD', 'O'), (')', ')', 'O')], [('365', 'CD', 'O'), ('Cats', 'NNPS', '

# Classifiers
Check whether stemming/lemmatization make difference in final classifiers
-> does it improve/worsen classifier?

## Split data into train/test

In order to split the data into train and test sets, we create a dataframe containing the tokenized titles. We then split the data into 80% for training and 20% for testing. 

In [24]:
# create dataframe containing tokenized titles and genres
tok_title = pd.DataFrame({0: tokenized_titles})
data_in = [tok_title[0], df1["genre"]]
headers = ["titles", "genres"]

data = pd.concat(data_in, axis=1, keys=headers)
print(data[:5])

                                              titles     genres
0               [moms, family, wall, calendar, 2016]  Calendars
1             [doug, the, pug, 2016, wall, calendar]  Calendars
2  [moleskine, 2016, weekly, notebook, 12m, large...  Calendars
3       [365, cats, color, pageaday, calendar, 2016]  Calendars
4         [sierra, club, engagement, calendar, 2016]  Calendars


In [65]:
# split data into train and test
import numpy as np

test_pct=0.2 # split into 80/20%

# create mask
mask = np.random.choice([0, 1], p=[1 - test_pct, test_pct], size=data.shape[0])

# apply mask
data["mask"] = mask
test = data[data["mask"] == 1]
train = data[data["mask"] == 0]

# removing column
test = test.drop("mask", axis="columns").reset_index()
train = train.drop("mask", axis="columns").reset_index()

# remove original indexing data (otherwise we have double indexing)
test = test.drop("index", axis="columns")
train = train.drop("index", axis="columns")

## Naive Bayes classifier

We build the Naive Bayes classifier from scratch. In order to do this, we create lists of the following: token frequencies in total vocabulary, token frequencies in each genre, number of words in each genre, number of words in total vocabulary, number of total titles, number of titles in each genre. Using these, we compute the priors for each genre and the likelihoods for each word to be in any genre. 
Using both the priors and the likelihoods, we create a dataframe with prediction values. By taking the highest prediction value of a particular word across the genres, we can get the predicted genre for that word.

We then test out whether the model works for the test data.

### Train model: Prep

In [46]:
from collections import Counter

In [47]:
# token frequencies within each title (we don't need this...)
unigram_count = []
for title in tokenized_titles:
    uni_title = Counter()
    for i in title:
        uni_title[i] += 1
    unigram_count.append(uni_title)

print(unigram_count[:5]) # test

[Counter({'moms': 1, 'family': 1, 'wall': 1, 'calendar': 1, '2016': 1}), Counter({'doug': 1, 'the': 1, 'pug': 1, '2016': 1, 'wall': 1, 'calendar': 1}), Counter({'moleskine': 1, '2016': 1, 'weekly': 1, 'notebook': 1, '12m': 1, 'large': 1, 'black': 1, 'soft': 1, 'cover': 1, '5': 1, 'x': 1, '825': 1}), Counter({'365': 1, 'cats': 1, 'color': 1, 'pageaday': 1, 'calendar': 1, '2016': 1}), Counter({'sierra': 1, 'club': 1, 'engagement': 1, 'calendar': 1, '2016': 1})]


In [48]:
# token frequencies in total vocab
tok_freq = Counter()
for title in train['titles']:
    for i in title:
        tok_freq[i] += 1
        
print(tok_freq)

Counter({'the': 92386, 'and': 54646, 'of': 51071, 'a': 31269, 'to': 27836, 'for': 21349, 'in': 21256, 'guide': 15989, 'edition': 12478, 'book': 8089, 'with': 7791, 'your': 7415, 'from': 6476, 'series': 6324, 'on': 6009, 'how': 5895, 'an': 5174, 'new': 4818, 'life': 4718, 'world': 4030, 'volume': 3985, 'history': 3982, 'you': 3856, '1': 3316, 'by': 3222, 'recipes': 3192, 'american': 3006, 'travel': 2915, 'complete': 2764, 'law': 2683, 'story': 2661, '2': 2648, 'art': 2631, 'handbook': 2537, 'health': 2535, 'my': 2489, 'books': 2396, 'science': 2269, 'at': 2230, 'calendar': 2224, 'best': 2062, 'novel': 2048, 'practice': 2003, 'i': 2001, 'guides': 1957, 'map': 1900, 'cookbook': 1888, 'that': 1872, 'vol': 1802, 'love': 1797, 'about': 1784, '2016': 1769, 'what': 1755, 'great': 1722, 'stories': 1713, 'business': 1693, 'war': 1646, 'is': 1643, 'more': 1632, 'it': 1594, 'introduction': 1569, 'one': 1565, 'modern': 1559, 'management': 1539, 'second': 1538, 'first': 1536, 'manual': 1524, 'americ

In [49]:
# group data by genre
# could probably be done MUCH nicer, but couldn't get for-loop to work...

grouped = train.groupby(train.genres)

Calendars = grouped.get_group("Calendars")
Comics = grouped.get_group("Comics & Graphic Novels")
Test = grouped.get_group("Test Preparation")
Mystery = grouped.get_group("Mystery, Thriller & Suspense")
SciFi = grouped.get_group("Science Fiction & Fantasy")
Romance = grouped.get_group("Romance")
Humor = grouped.get_group("Humor & Entertainment")
Literature = grouped.get_group("Literature & Fiction")
LGBTQ = grouped.get_group("Gay & Lesbian")
Engineering = grouped.get_group("Engineering & Transportation")
Food = grouped.get_group("Cookbooks, Food & Wine")
Crafts = grouped.get_group("Crafts, Hobbies & Home")
Arts = grouped.get_group("Arts & Photography")
Education = grouped.get_group("Education & Teaching")
Parenting = grouped.get_group("Parenting & Relationships")
SelfHelp = grouped.get_group("Self-Help")
Computers = grouped.get_group("Computers & Technology")
Medical = grouped.get_group("Medical Books")
Science = grouped.get_group("Science & Math")
Health = grouped.get_group("Health, Fitness & Dieting")
Business = grouped.get_group("Business & Money")
Law = grouped.get_group("Law")
Biographies = grouped.get_group("Biographies & Memoirs")
History = grouped.get_group("History")
Politics = grouped.get_group("Politics & Social Sciences")
Reference = grouped.get_group("Reference")
Bibles = grouped.get_group("Christian Books & Bibles")
Religion = grouped.get_group("Religion & Spirituality")
Sports = grouped.get_group("Sports & Outdoors")
Teen = grouped.get_group("Teen & Young Adult")
Childrens = grouped.get_group("Children's Books")
Travel = grouped.get_group("Travel")

GenreGroups = [Calendars['titles'], Comics['titles'], Test['titles'], Mystery['titles'], SciFi['titles'], 
               Romance['titles'], Humor['titles'], Literature['titles'], LGBTQ['titles'], Engineering['titles'], 
               Food['titles'], Crafts['titles'], Arts['titles'], Education['titles'], Parenting['titles'], 
               SelfHelp['titles'], Computers['titles'], Medical['titles'], Science['titles'], Health['titles'], 
               Business['titles'], Law['titles'], Biographies['titles'], History['titles'], Politics['titles'], 
               Reference['titles'], Bibles['titles'], Religion['titles'], Sports['titles'], Teen['titles'], 
               Childrens['titles'], Travel['titles']]

In [50]:
# token frequencies in each genre
genre_count = []
for g in GenreGroups:
    genre_title = Counter()
    for title in g:
        for i in title:
            genre_title[i] += 1
    genre_title = dict(genre_title)
    genre_count.append(genre_title)

In [51]:
# token frequencies in each genre
genre_count[0] # genres in same order as GenreGroups

{'moms': 8,
 'family': 11,
 'wall': 812,
 'calendar': 1993,
 '2016': 1127,
 'doug': 1,
 'the': 374,
 'pug': 4,
 '365': 78,
 'cats': 39,
 'color': 12,
 'pageaday': 60,
 'sierra': 2,
 'club': 4,
 'engagement': 67,
 'wilderness': 8,
 'thomas': 11,
 'kinkade': 9,
 'disney': 22,
 'dreams': 5,
 'collection': 9,
 'dilbert': 4,
 'daytoday': 93,
 'mary': 8,
 'engelbreit': 6,
 'deluxe': 33,
 'never': 2,
 'give': 1,
 'up': 10,
 'cat': 28,
 'gallery': 14,
 'llewellyns': 6,
 'witches': 3,
 'datebook': 1,
 'amy': 3,
 'knapp': 2,
 'big': 8,
 'grid': 2,
 'essential': 1,
 'organization': 1,
 'and': 142,
 'communication': 1,
 'tool': 1,
 'for': 57,
 'entire': 1,
 'outlander': 2,
 'audubon': 7,
 'nature': 12,
 'national': 14,
 'park': 2,
 'foundation': 1,
 'your': 14,
 'year': 57,
 'mindful': 2,
 'coloring': 14,
 'through': 26,
 'seasons': 4,
 'enjoy': 1,
 'joy': 1,
 'moleskine': 29,
 'weekly': 57,
 'notebook': 17,
 '12m': 7,
 'extra': 4,
 'large': 26,
 'black': 27,
 'soft': 7,
 'cover': 32,
 '75': 4,
 '

In [52]:
# number of words in a class
len(genre_count[0]) # first genre

2381

In [53]:
# number of total vocabulary (training set)
V = len(Counter(tok_freq))
print(V)

74831


In [54]:
# number of titles (in training set)
N_titles = len(train)
print(N_titles)

166130


In [55]:
# number of titles in each genre
N_genre = train['genres'].value_counts()
print(N_genre[:5])

Travel                       14641
Children's Books             10938
Medical Books                 9701
Health, Fitness & Dieting     9560
Business & Money              7970
Name: genres, dtype: int64


### Priors

In [56]:
# priors of each genre (probability of title being in specific genre)
prob_title = []
for g in range(len(N_genre)):
    prob = N_genre[g] / N_titles
    prob_title.append(prob)

zipped_values = zip(df1.genre.unique(), prob_title)
prior = list(zipped_values)

print(prior[:5]) # test

[('Calendars', 0.08812977788478903), ('Comics & Graphic Novels', 0.06584000481550593), ('Test Preparation', 0.058394028772647925), ('Mystery, Thriller & Suspense', 0.05754529585264552), ('Science Fiction & Fantasy', 0.04797447781857581)]


### Likelihoods

In [57]:
# likelihoods of each word (probability of word being in specific genre)

# create empty dataframe
likelihood = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq))

len_gc = -1
for i in genre_count: # loop through genres
    len_gc += 1 # create genre index
    for word in i: 
        p = (genre_count[len_gc][word] + 1) / (len(genre_count[len_gc]) + V) # probability
        likelihood.loc[word, likelihood.columns[len_gc]] = p # replace NaN in dataframe with p

In [58]:
# now fill likelihoods of words that haven't appeared in genre

len_gc = -1
for c, v in likelihood.iteritems(): # loop through genres in dataframe
  len_gc += 1 # create genre index
  p = 1 / (len(genre_count[len_gc]) + V) # probability of word not appearing in that genre
  likelihood[c].fillna(p, inplace=True) # replace remaining NaNs in dataframe with p

In [59]:
likelihood

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,0.000117,0.000025,0.000013,0.000013,0.000013,0.000025,0.000072,0.000012,0.000052,0.000013,0.000132,0.000035,0.000036,0.000039,0.000304,0.000025,0.000012,0.000024,0.000012,0.000104,0.000036,0.000024,0.000024,0.000012,0.000025,0.000025,0.000096,0.000036,0.000012,0.000024,0.000047,0.000045
family,0.000155,0.000204,0.000219,0.000103,0.000050,0.000453,0.000453,0.000421,0.000412,0.000075,0.002332,0.000398,0.000253,0.000154,0.001673,0.000215,0.000110,0.000591,0.000142,0.001217,0.000748,0.001132,0.000803,0.000355,0.000224,0.000326,0.000914,0.000346,0.000121,0.000337,0.000595,0.001101
wall,0.010529,0.000013,0.000026,0.000051,0.000050,0.000013,0.000227,0.000036,0.000013,0.000025,0.000024,0.000211,0.000289,0.000026,0.000013,0.000025,0.000037,0.000059,0.000107,0.000023,0.000760,0.000085,0.000110,0.000118,0.000037,0.000176,0.000108,0.000167,0.000134,0.000084,0.000187,0.000420
calendar,0.025825,0.000013,0.000013,0.000013,0.000013,0.000025,0.000322,0.000036,0.000013,0.000151,0.000048,0.000281,0.000530,0.000013,0.000051,0.000051,0.000086,0.000012,0.000166,0.000046,0.000047,0.000012,0.000012,0.000012,0.000050,0.000038,0.000108,0.000299,0.000170,0.000024,0.000035,0.000443
2016,0.014609,0.000013,0.001365,0.000013,0.000025,0.000013,0.000346,0.000072,0.000039,0.000452,0.000060,0.000445,0.000554,0.000193,0.000114,0.000114,0.000526,0.000567,0.000130,0.000081,0.000321,0.000061,0.000012,0.000036,0.000062,0.000313,0.000156,0.000287,0.000182,0.000012,0.000117,0.001453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ballrooms,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
riverboat,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
historymapped,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023
1790,0.000013,0.000013,0.000013,0.000013,0.000013,0.000013,0.000012,0.000012,0.000013,0.000013,0.000012,0.000012,0.000012,0.000013,0.000013,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000012,0.000013,0.000012,0.000012,0.000012,0.000012,0.000012,0.000023


In [60]:
# predictions on word by word basis using words in titles of test data

predictors = pd.DataFrame(columns=df1.genre.unique(), index=dict.keys(tok_freq)) # create empty dataframe
len_gc = -1
for i in prior:
  len_gc += 1 # create genre index
  # square all likelihoods then multiply them each with prior of that genre
  predictors = likelihood.loc[:, df1.genre.unique().tolist()] ** 2 # square likelihoods
  predictors = predictors.loc[:, df1.genre.unique().tolist()] * np.array(prior[len_gc][1]) # multiply by priors

In [61]:
predictors

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,8.783581e-11,4.194797e-12,1.071724e-12,1.067899e-12,1.025478e-12,4.094069e-12,3.309477e-11,9.338300e-13,1.715024e-11,1.018666e-12,1.130641e-10,7.992390e-12,8.449498e-12,9.614053e-12,5.984431e-10,4.144918e-12,9.686464e-13,3.608199e-12,9.073796e-13,7.034627e-11,8.198437e-12,3.832251e-12,3.829826e-12,9.074226e-13,4.018479e-12,4.063844e-12,5.988453e-11,8.298413e-12,9.537619e-13,3.749456e-12,1.408001e-11,1.332433e-11
family,1.561526e-10,2.684670e-10,3.097284e-10,6.834551e-11,1.640764e-11,1.326478e-09,1.327468e-09,1.143942e-09,1.097615e-09,3.667199e-11,3.516760e-08,1.026578e-09,4.140254e-10,1.538249e-10,1.810290e-08,2.994703e-10,7.846036e-11,2.255124e-09,1.306627e-10,9.574909e-09,3.615511e-09,8.286285e-09,4.170681e-09,8.166803e-10,3.254968e-10,6.867896e-10,5.404579e-09,7.754406e-10,9.537619e-11,7.348935e-10,2.288882e-09,7.835541e-09
wall,7.167500e-07,1.048699e-12,4.286898e-12,1.708638e-11,1.640764e-11,1.023517e-12,3.318670e-10,8.404470e-12,1.071890e-12,4.074666e-12,3.737655e-12,2.877260e-10,5.407679e-10,4.272913e-12,1.038964e-12,4.144918e-12,8.717817e-12,2.255124e-11,7.349775e-11,3.473890e-12,3.731200e-09,4.694507e-11,7.755398e-11,9.074226e-11,9.041578e-12,1.991284e-10,7.579136e-11,1.807210e-10,1.154052e-10,4.593084e-11,2.252802e-10,1.140063e-09
calendar,4.311585e-06,1.048699e-12,1.071724e-12,1.067899e-12,1.025478e-12,4.094069e-12,6.701691e-10,8.404470e-12,1.071890e-12,1.466880e-10,1.495062e-11,5.115130e-10,1.817581e-09,1.068228e-12,1.662342e-11,1.657967e-11,4.746367e-11,9.020497e-13,1.778464e-10,1.389556e-11,1.457500e-11,9.580627e-13,9.574565e-13,9.074226e-13,1.607392e-11,9.143649e-12,7.579136e-11,5.762787e-10,1.869373e-10,3.749456e-12,7.920008e-12,1.266645e-09
2016,1.379764e-06,1.048699e-12,1.204190e-08,1.067899e-12,4.101910e-12,1.023517e-12,7.731306e-10,3.361788e-11,9.647011e-12,1.320192e-09,2.336035e-11,1.282335e-09,1.986571e-09,2.403513e-10,8.415606e-11,8.393459e-11,1.791027e-09,2.078323e-09,1.097929e-10,4.255515e-11,6.640734e-10,2.395157e-11,9.574565e-13,8.166803e-12,2.511549e-11,6.349756e-10,1.581326e-10,5.310984e-10,2.145964e-10,9.373641e-13,8.800009e-11,1.364412e-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ballrooms,1.084393e-12,1.048699e-12,1.071724e-12,1.067899e-12,1.025478e-12,1.023517e-12,9.192991e-13,9.338300e-13,1.071890e-12,1.018666e-12,9.344139e-13,8.880433e-13,9.388332e-13,1.068228e-12,1.038964e-12,1.036230e-12,9.686464e-13,9.020497e-13,9.073796e-13,8.684725e-13,9.109375e-13,9.580627e-13,9.574565e-13,9.074226e-13,1.004620e-12,1.015961e-12,9.356958e-13,9.220459e-13,9.537619e-13,9.373641e-13,8.800009e-13,3.331084e-12
riverboat,1.084393e-12,1.048699e-12,1.071724e-12,1.067899e-12,1.025478e-12,1.023517e-12,9.192991e-13,9.338300e-13,1.071890e-12,1.018666e-12,9.344139e-13,8.880433e-13,9.388332e-13,1.068228e-12,1.038964e-12,1.036230e-12,9.686464e-13,9.020497e-13,9.073796e-13,8.684725e-13,9.109375e-13,9.580627e-13,9.574565e-13,9.074226e-13,1.004620e-12,1.015961e-12,9.356958e-13,9.220459e-13,9.537619e-13,9.373641e-13,8.800009e-13,3.331084e-12
historymapped,1.084393e-12,1.048699e-12,1.071724e-12,1.067899e-12,1.025478e-12,1.023517e-12,9.192991e-13,9.338300e-13,1.071890e-12,1.018666e-12,9.344139e-13,8.880433e-13,9.388332e-13,1.068228e-12,1.038964e-12,1.036230e-12,9.686464e-13,9.020497e-13,9.073796e-13,8.684725e-13,9.109375e-13,9.580627e-1

In [62]:
predictors.loc[["moms"]]

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
moms,8.783581e-11,4.194797e-12,1.071724e-12,1.067899e-12,1.025478e-12,4.094069e-12,3.309477e-11,9.338300e-13,1.715024e-11,1.018666e-12,1.130641e-10,7.992390e-12,8.449498e-12,9.614053e-12,5.984431e-10,4.144918e-12,9.686464e-13,3.608199e-12,9.073796e-13,7.034627e-11,8.198437e-12,3.832251e-12,3.829826e-12,9.074226e-13,4.018479e-12,4.063844e-12,5.988453e-11,8.298413e-12,9.537619e-13,3.749456e-12,1.408001e-11,1.332433e-11


In [63]:
# output genre that has highest prediction value for input word
maxValueIndexObj = predictors.idxmax(axis=1)
print(list(maxValueIndexObj[["high"]]))
print(list(maxValueIndexObj[["higher"]]))
print(list(maxValueIndexObj[["highest"]]))

['Health, Fitness & Dieting']
['Education & Teaching']
['Sports & Outdoors']


### Test model

In [66]:
# function that will return list of predicted genres based on title input
# function is very slow -> OPTIMIZE (otherwise takes 11.5hrs for test dataset...)
def pred_genre(title):
  prediction = []
  for w in title:
    ## CONDITION: if word exists in total vocabulary
    if w in predictors.index:
      maxValueIndexObj = predictors.idxmax(axis=1)
      prediction += list(maxValueIndexObj[[w]])
    ## CONDITION: if word does not exist in total vocabulary
    if w not in predictors.index:
      pass

  # probability of predicted genre being chosen
  d = dict(Counter(prediction))
  d1 = {k: v / total for total in (sum(d.values()),) for k, v in d.items()}

  # output most likely predicted genre
  if prediction == []:
    return 'There is no prediction for this title.'
  else:
    return max(d1, key=d1.get) 
  
  # what if there is no maximum? (all equally predicted)
  # gives first predicted genre as output...

In [76]:
# predicted genre for first test title
import time
time_start = time.clock()

pred_genre(test['titles'][6000])

time_elapsed = (time.clock() - time_start)
print(time_elapsed)

0.7275489999992715


In [ ]:
pred_genre(test['titles'][678])

'There is no prediction for this title.'

In [68]:
# "real" genre of first test title
test['genres'][0]

'Calendars'

In [69]:
# titles that would give no prediction:
# these titles are one word long and not included in training set
for t in test['titles']:
  if len(t) == 1:
    for w in t:
      if w not in list(tok_freq.keys()):
        print(t)
  else:
    pass

['hicksville']
['swipe']
['agroland']
['selphelf']
['beatlesongs']
['parsifal']
['plainsong']
['greenbeard']
['kirakira']
['bodymap']
['lightningbolt']
['unicycling']
['obasan']
['swimmy']
['motorcyles']
['vietnamlaoscambodia']
['kanaval']


In [78]:
# this function will take 5 hours!!
predicted_genre = []
for t in test['titles']:
  predicted_genre.append(pred_genre(t))

KeyboardInterrupt: ignored

In [80]:
len(predicted_genre)

10619

In [75]:
# change list to dataframe to enable saving as csv file
df2 = pd.DataFrame(predicted_genre)
df2.columns = ['Predicted Genre']

# create and save csv file in Google Drive
df2.to_csv('/drive/My Drive/predicted_genre.csv', index=False)

In [76]:
# read csv file as pandas Dataframe
predicted_genre = pd.read_csv('/drive/My Drive/predicted_genre.csv',encoding='latin1')

In [ ]:
# create dataframe with titles, genres, predicted genres

### Accuracy of model

In [ ]:
# create confusion matrix
confusion = pd.DataFrame(columns=df1.genre.unique(), index=df1.genre.unique()) # create empty dataframe

TruePos = 0
# fill dataframe
for t in range(len(test)):
  if pred_genre(test['titles'][t]) == 'Calendars':
    if test['genres'][t] == 'Calendars':
      TruePos +=1
  # if statements for each genre
  else:
    pass

confusion.loc['Calendars', 'Calendars'] = TruePos

KeyboardInterrupt: ignored

In [ ]:
confusion

,Calendars,Comics & Graphic Novels,Test Preparation,"Mystery, Thriller & Suspense",Science Fiction & Fantasy,Romance,Humor & Entertainment,Literature & Fiction,Gay & Lesbian,Engineering & Transportation,"Cookbooks, Food & Wine","Crafts, Hobbies & Home",Arts & Photography,Education & Teaching,Parenting & Relationships,Self-Help,Computers & Technology,Medical Books,Science & Math,"Health, Fitness & Dieting",Business & Money,Law,Biographies & Memoirs,History,Politics & Social Sciences,Reference,Christian Books & Bibles,Religion & Spirituality,Sports & Outdoors,Teen & Young Adult,Children's Books,Travel
Calendars,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Comics & Graphic Novels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Test Preparation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Mystery, Thriller & Suspense",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Science Fiction & Fantasy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Romance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Humor & Entertainment,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Literature & Fiction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Gay & Lesbian,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Engineering & Transportation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# true positives -> use confusion matrix
TruePos = {'Calendars': 0, 'Comics & Graphic Novels': 0, 'Test Preparation': 0,
       'Mystery, Thriller & Suspense': 0, 'Science Fiction & Fantasy': 0,
       'Romance': 0, 'Humor & Entertainment': 0, 'Literature & Fiction': 0,
       'Gay & Lesbian': 0, 'Engineering & Transportation': 0,
       'Cookbooks, Food & Wine': 0, 'Crafts, Hobbies & Home': 0,
       'Arts & Photography': 0, 'Education & Teaching': 0,
       'Parenting & Relationships': 0, 'Self-Help': 0, 'Computers & Technology': 0,
       'Medical Books': 0, 'Science & Math': 0, 'Health, Fitness & Dieting': 0,
       'Business & Money': 0, 'Law': 0, 'Biographies & Memoirs': 0, 'History': 0,
       'Politics & Social Sciences': 0, 'Reference': 0,
       'Christian Books & Bibles': 0, 'Religion & Spirituality': 0,
       'Sports & Outdoors': 0, 'Teen & Young Adult': 0, "Children's Books": 0,
       'Travel': 0}

for t in range(len(test)):
  if pred_genre(test['titles'][t]) == 'Calendars' and test['genres'][t] == 'Calendars':
    TruePos['Calendars'] +=1
  #elif # other genres
  else:
    pass

# create dictionary for true positives
# if e.g. Calendars == Calendars -> add 1 to key 'Calendars' etc. for each genre

KeyboardInterrupt: ignored

In [ ]:
# false positives -> use confusion matrix

In [ ]:
# false negatives -> use confusion matrix

In [ ]:
# precision = TruePos / (TruePos + FalsePos)

In [ ]:
# recall = TruePos / (TruePos + FalseNeg)

In [ ]:
# macroaveraging precision + recall
# average of precisions and recalls across all genres

In [ ]:
# microaveraging precision + recall
# all true pos added together, all false pos, all false negs
# then get precision + recall values

In [ ]:
# F1: weighted mean of precision and recall
# using micro/macroaveraging (?)

F1 = (2 * precision * recall) / (precision + recall)